# CAPSTONE PROJECT 2: A COMPUTER VISION MODEL WHICH DETECTS BONE FRACTURES IN THE UPPER EXTREMITIES NAMELY: WRISTS, FOREARMS, UPPER ARM, & SHOULDER FRACTURES
#### Modeling



***
Goal: Build two to three different models and identify the best one. <br>
- Fit your models with a training dataset<bf>
Hint: Try a number of different models: you will want to compare their outputs in the
model evaluation stage. For example, if you’re writing a classification model, you should
implement both an entropy model and a Gini impurity model. For hyperparameter tuning,
think of methods like cross-validation.<br>
- Review model outcomes — Iterate over additional models as needed<br>
Hint: you may want to use standard model evaluation metrics such as accuracy, recall,
precision, and F1.<br>
- Identify the final model that you think is the best model for this project<br>
Hint: the most powerful model isn’t always the best one to use. Other considerations
include computational complexity, scalability, and maintenance costs.<br>
Review the following questions and apply them to your analysis:<br>
- Does my data involve a time series or forecasting? If so, am I splitting the train
and test data appropriately?<br>
- Is my response variable continuous or categorical?<br>
***

In [14]:
# Import modules:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from random import sample

# Image Analysis imports
import sklearn as skl
import skimage as ski
import cv2
from skimage.filters import sobel
from skimage import exposure
from skimage.filters import try_all_threshold
from skimage.filters import threshold_yen
from skimage import measure
from skimage.segmentation import slic

# PyTorch Analysis imports
import torch
from torch import nn
from tqdm.auto import tqdm
from torchvision import transforms
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import albumentations as A
import torchvision
from torchvision.utils import draw_bounding_boxes, draw_segmentation_masks
import torchvision.transforms.functional as F
from torchvision.transforms import v2

# Feature extraction imports
from torchvision.models import resnet50
from torchvision.models.feature_extraction import get_graph_node_names
from torchvision.models.feature_extraction import create_feature_extractor
from torchvision.models.detection.mask_rcnn import MaskRCNN
from torchvision.models.detection.backbone_utils import LastLevelMaxPool
from torchvision.ops.feature_pyramid_network import FeaturePyramidNetwork
from torch.utils.data import Dataset
from PIL import Image

In [6]:
#Import the data as Pandas DataFrames: df_test, df_train, df_val
data_links = [r'C:\Users\micha\OneDrive\Documents\GitHub\SpringBoardDataScience\Capstone_Project_2\Data\df_test.csv', 
              r'C:\Users\micha\OneDrive\Documents\GitHub\SpringBoardDataScience\Capstone_Project_2\Data\df_train.csv', 
              r'C:\Users\micha\OneDrive\Documents\GitHub\SpringBoardDataScience\Capstone_Project_2\Data\df_val.csv']

def multi_df(list):
    temp_list = []
    for i in list:
        temp_list.append(pd.read_csv(i))
    return temp_list
    
df_test, df_train, df_val = multi_df(data_links)

# Trim the extra index column
df_train.drop(columns='Unnamed: 0', inplace=True)
df_test.drop(columns='Unnamed: 0', inplace=True)
df_val.drop(columns='Unnamed: 0', inplace=True)


In [15]:
# GLOBALS
# Project Main Directory path
dir_path = r"C:\Users\micha\OneDrive\Documents\GitHub\SpringBoardDataScience\Capstone_Project_2"

# Data Paths
train_dir_path = r"C:\Users\micha\OneDrive\Documents\GitHub\SpringBoardDataScience\Capstone_Project_2\Data\train"
val_dir_path = r"C:\Users\micha\OneDrive\Documents\GitHub\SpringBoardDataScience\Capstone_Project_2\Data\valid"
test_dir_path = r"C:\Users\micha\OneDrive\Documents\GitHub\SpringBoardDataScience\Capstone_Project_2\Data\test"

# Image Scalars
Width = 400
Height = 400

# Labels
num_classes = 8
classes=['elbow positive', 'fingers positive', 'forearm fracture', 'humerus fracture', 'humerus', 'shoulder fracture', 'wrist positive', 'no fracture']
c2l={k:v for k,v in list(zip(classes,list(range(num_classes))))}
l2c={v:k for k,v in c2l.items()}